# Clustering Crypto

In [381]:
# Initial imports
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [382]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [383]:
# Create a DataFrame
response = requests.get(url)
response_json = response.json()

In [384]:
crypto_df = pd.DataFrame(response_json['Data']).transpose()
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,0.000244,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [385]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply'
crypto_df.drop(crypto_df.columns.difference([
    'CoinName', 
    'Algorithm', 
    'IsTrading', 
    'ProofType', 
    'TotalCoinsMined', 
    'MaxSupply',
]), axis='columns', inplace=True)

crypto_df.head()

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,41.999952,42
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,-1
404,404Coin,Scrypt,PoW/PoS,True,0,-1
433,433 Token,N/A,N/A,False,NaN,NaN


In [386]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [387]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [388]:
# Remove the "IsTrading" column
crypto_df.drop(columns='IsTrading', inplace=True)

In [389]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]

In [390]:
# Drop rows where there are 'N/A' text values
crypto_df.replace('N/A', np.nan, inplace=True)
crypto_df.dropna(inplace=True)

In [391]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name = crypto_df['CoinName'].to_frame()

In [392]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns='CoinName', inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,Scrypt,PoW/PoS,4.199995e+01,42.0
NSR,PoS,PoS,6.178385e+09,0.0
TRI,X13,PoW/PoS,1.916272e+05,0.0
CMTC,Scrypt,PoW,8.728300e+05,0.0
CHAT,Scrypt,PoW/PoS,1.000000e+09,-1.0


In [393]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,MaxSupply,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP-20 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,Algorithm_Blake2b,Algorithm_C31,...,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_ProgPoW/PoS,ProofType_Proof of Authority,ProofType_Proof-of-Work,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW,ProofType_dPoW/PoW
42,4.199995e+01,42.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NSR,6.178385e+09,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRI,1.916272e+05,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CMTC,8.728300e+05,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHAT,1.000000e+09,-1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [394]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
X_scaled[0]

array([-0.08660384, -0.09087225, -0.08481889, -0.08481889, -0.08481889,
       -0.12038585, -0.08481889, -0.08481889, -0.12038585, -0.12038585,
       -0.14797909, -0.08481889, -0.08481889, -0.08481889, -0.24618298,
       -0.12038585, -0.08481889, -0.08481889, -0.08481889, -0.29201253,
       -0.08481889, -0.08481889, -0.24618298, -0.08481889, -0.08481889,
       -0.12038585, -0.08481889, -0.08481889, -0.08481889, -0.08481889,
       -0.08481889, -0.08481889, -0.14797909, -0.08481889, -0.08481889,
       -0.12038585, -0.19245009, -0.08481889, -0.08481889, -0.14797909,
       -0.12038585, -0.29201253, -0.12038585, -0.08481889, -0.08481889,
       -0.08481889,  2.19848433, -0.08481889, -0.08481889, -0.08481889,
       -0.08481889, -0.08481889, -0.21160368, -0.08481889, -0.19245009,
       -0.12038585, -0.08481889, -0.08481889, -0.08481889, -0.08481889,
       -0.08481889, -0.26211122, -0.08481889, -0.08481889, -0.12038585,
       -0.12038585, -0.08481889, -0.31994094, -0.08481889, -0.08

### Reducing Dimensions Using PCA

In [395]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [396]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=pca.fit_transform(X_scaled), 
    columns=['PC1', 'PC2', 'PC3'], 
    index=crypto_df.index,
)
pcs_df.sort_index(axis=1, ascending=True).head()

,PC1,PC2,PC3
42,0.222851,-1.326062,-1.343659
NSR,0.696263,-1.169298,-0.312068
TRI,0.654794,-1.974567,-1.626911
CMTC,-0.853522,0.447471,-0.364627
CHAT,0.222855,-1.326058,-1.343659


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [397]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create the Elbow Curve using hvPlot
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [398]:
elbow_data

{'k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'inertia': [940.8537753687656,
  642.5301648261744,
  392.426088907011,
  181.17813516617665,
  73.20954429582267,
  53.797103663474324,
  37.989377254349186,
  22.797795489851218,
  17.574981348162957,
  12.323166189482732]}

Running K-Means with `k=<your best value for k here>`

In [403]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat(
    [coin_name, crypto_df, pcs_df],
    join='inner', 
    axis='columns'
)
clustered_df['Class'] = predictions
clustered_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC1,PC2,PC3,Class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42.0,0.222851,-1.326062,-1.343659,0
NSR,NuShares,PoS,PoS,6.178385e+09,0.0,0.696263,-1.169298,-0.312068,0
TRI,Triangles Coin,X13,PoW/PoS,1.916272e+05,0.0,0.654794,-1.974567,-1.626911,0
CMTC,CometCoin,Scrypt,PoW,8.728300e+05,0.0,-0.853522,0.447471,-0.364627,1
CHAT,OpenChat,Scrypt,PoW/PoS,1.000000e+09,-1.0,0.222855,-1.326058,-1.343659,0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [405]:
# Scale data to create the scatter plot
clustered_encoded = pd.get_dummies(clustered_df)
clustered_scaled = StandardScaler().fit_transform(clustered_encoded)

In [407]:
# Plot the scatter with x="TotalCoinsMined" and y="MaxSupply"
clustered_scaled.hvplot.scatter(x='TotalCoinsMined', y='MaxSupply', by="Class")


AttributeError: 'numpy.ndarray' object has no attribute 'hvplot'

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies
